# AWS Glue Studio Notebook - Estoques

##### Você está executando um Notebook do AWS Glue Studio. Para começar a utilizá-lo, você precisa iniciar um Sessão interativa do AWS Glue.

Este é o notebook utilizado no processo de ETL dos dados da Tabela Estoques da Zoop. Observe cada uma das etapas e, em caso de dúvidas, assista ao vídeo correspondente ao tratamento desta tabela.

Faça bom proveito deste conteúdo! &#x1F642;

#### Opcional: Execute esta célula se quiser observar os comandos deste notebook ("magics").

In [1]:
%help

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 


#### Configurando e iniciando uma sessão interativa.

In [1]:
%idle_timeout 60
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Current idle_timeout is None minutes.
idle_timeout has been set to 60 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 60
Session ID: ef2c5254-c35e-4519-b7c0-7b3e0b25ffe1
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
Waiting for session ef2c5254-c35e-4519-b7c0-7b3e0b25ffe1 to get into ready status...
Session ef2c5254-c35e-4519-b7c0-7b3e0b25ffe1 has been created.



#### Criando um DynamicFrame de uma tabela no AWS Glue Data Catalog e mostrando seu schema

In [2]:
estoques_zoop_dyf = glueContext.create_dynamic_frame.from_catalog(database='db-glue-zoop', table_name='zoop-glue-estoques_zoop_bronze_parquet')
estoques_zoop_dyf.printSchema()

root
|-- ID_estoque: long
|-- ID_produto: long
|-- Produto: string
|-- Categoria_produto: string
|-- Data: string
|-- Horario: string
|-- Quantidade_em_estoque: long
|-- Quantidade_novos_produtos: long
|-- Quantidade_vendida: long


#### Criando um Spark DataFrame de um DynamicFrame

In [3]:
estoques_zoop_df = estoques_zoop_dyf.toDF()
estoques_zoop_df.show(5)

+----------+----------+---------------+-----------------+----------+--------+---------------------+-------------------------+------------------+
|ID_estoque|ID_produto|        Produto|Categoria_produto|      Data| Horario|Quantidade_em_estoque|Quantidade_novos_produtos|Quantidade_vendida|
+----------+----------+---------------+-----------------+----------+--------+---------------------+-------------------------+------------------+
|       535|        25|Panela_elétrica| Eletrodomésticos|2022/12/01|21:58:00|                  319|                       14|                16|
|      1020|         0|   Smart_TV_55"|      Eletrônicos|2024/05/01|09:14:00|                  389|                       11|                28|
|       810|         0|   Smart_TV_55"|      Eletrônicos|2023/10/01|12:15:00|                  355|                       19|                23|
|       699|         9|Aspirador_de_pó| Eletrodomésticos|2023/06/01|20:09:00|                  250|                       50|     

#### Tratando a coluna `Produto` removendo (_) que separam os nomes dos produtos

In [4]:
from pyspark.sql.functions import regexp_replace

estoques_zoop_df = estoques_zoop_df.withColumn("Produto", regexp_replace("Produto", "_", " "))
estoques_zoop_df.show(3)

+----------+----------+---------------+-----------------+----------+--------+---------------------+-------------------------+------------------+
|ID_estoque|ID_produto|        Produto|Categoria_produto|      Data| Horario|Quantidade_em_estoque|Quantidade_novos_produtos|Quantidade_vendida|
+----------+----------+---------------+-----------------+----------+--------+---------------------+-------------------------+------------------+
|       535|        25|Panela elétrica| Eletrodomésticos|2022/12/01|21:58:00|                  319|                       14|                16|
|      1020|         0|   Smart TV 55"|      Eletrônicos|2024/05/01|09:14:00|                  389|                       11|                28|
|       810|         0|   Smart TV 55"|      Eletrônicos|2023/10/01|12:15:00|                  355|                       19|                23|
+----------+----------+---------------+-----------------+----------+--------+---------------------+-------------------------+-----

#### Ajustando as datas da coluna `Data`

In [5]:
from pyspark.sql.functions import to_date

estoques_zoop_df = estoques_zoop_df.withColumn("Data", to_date("Data", "yyyy/MM/dd"))
estoques_zoop_df.show(3)

+----------+----------+---------------+-----------------+----------+--------+---------------------+-------------------------+------------------+
|ID_estoque|ID_produto|        Produto|Categoria_produto|      Data| Horario|Quantidade_em_estoque|Quantidade_novos_produtos|Quantidade_vendida|
+----------+----------+---------------+-----------------+----------+--------+---------------------+-------------------------+------------------+
|       535|        25|Panela elétrica| Eletrodomésticos|2022-12-01|21:58:00|                  319|                       14|                16|
|      1020|         0|   Smart TV 55"|      Eletrônicos|2024-05-01|09:14:00|                  389|                       11|                28|
|       810|         0|   Smart TV 55"|      Eletrônicos|2023-10-01|12:15:00|                  355|                       19|                23|
+----------+----------+---------------+-----------------+----------+--------+---------------------+-------------------------+-----

#### Convertendo um Spark Data Frame em um DynamicFrame

In [6]:
from awsglue.dynamicframe import DynamicFrame

estoques_zoop_dyf = DynamicFrame.fromDF(estoques_zoop_df, glueContext)

#### Ajustando os tipos dos dados e mapeando as colunas desejadas

In [7]:
estoques_zoop_dyf_mapeado = estoques_zoop_dyf.apply_mapping(
    mappings=[
        ("ID_estoque", "long", "id_estoque", "long"),
        ("ID_produto", "long", "id_produto", "long"),
        ("Produto", "string", "produto", "string"),
        ("Categoria_produto", "string", "categoria_produto", "string"),
        ("Data", "date", "data", "date"),
        ("Horario", "string", "horario", "timestamp"),
        ("Quantidade_em_estoque", "long", "quantidade_em_estoque", "int"),
        ("Quantidade_novos_produtos", "long", "quantidade_novos_produtos", "int"),
        ("Quantidade_vendida", "long", "quantidade_vendida", "int")
    ]
)

estoques_zoop_dyf_mapeado.printSchema()

root
|-- id_estoque: long
|-- id_produto: long
|-- produto: string
|-- categoria_produto: string
|-- data: date
|-- horario: timestamp
|-- quantidade_em_estoque: int
|-- quantidade_novos_produtos: int
|-- quantidade_vendida: int


#### Escrevendo os dados do DynamicFrame em um objeto do Bucket S3 na camada silver e em uma tabela no AWS Glue Data Catalog


In [8]:
s3output = glueContext.getSink(
  path="s3://dados-zoop/silver/estoques",  # adicione o nome do seu bucket no path
  connection_type="s3",
  updateBehavior="UPDATE_IN_DATABASE",
  partitionKeys=[],
  compression="snappy",
  enableUpdateCatalog=True,
  transformation_ctx="s3output",
)
s3output.setCatalogInfo(
  catalogDatabase="db-glue-zoop", catalogTableName="zoop-glue-estoques_zoop_silver"
)
s3output.setFormat("glueparquet")
s3output.writeFrame(estoques_zoop_dyf_mapeado)